## Storing Things

In [1]:
my_data = {'project': 'insurance_payments',
           'payment_values': [1000, 1500, 1200]}

In [2]:
with open('tmp_file', 'w') as data_file:
    data_file.write(str(my_data))

In [4]:
recovered_data = {}
with open('tmp_file') as data_file:
    recovered_data = eval(data_file.read())
recovered_data

{'payment_values': [1000, 1500, 1200], 'project': 'insurance_payments'}

# Don't try this at home

What if your file contains:

     import subprocess
     subprocess.call(['rm', '-rf', '~'])

Eval will mess you up!

# More limitations

In [5]:
class DummyClass(object):
    def __init__(self, n):
        self.n = n

In [6]:
my_data = [DummyClass(i) for i in range(10)]

In [7]:
eval(str(my_data))

SyntaxError: invalid syntax (<string>, line 1)

# Pickle

In [8]:
import pickle

In [10]:
with open('tmp_file', 'w') as data_file:
    pickle.dump(my_data, data_file)

In [14]:
recovered_data = []
with open('tmp_file') as data_file:
    recovered_data = pickle.load(data_file)
recovered_data

# Databases

Databases are places to permanently store data.

## Installation

### Mac

    brew update
    brew install mongodb
    brew services start mongodb

### Linux

    apt install mongodb

or

    yum install mongodb

https://docs.mongodb.com/manual/reference/operator/query/

In [1]:
from pymongo import MongoClient
from datetime import datetime

In [2]:
MongoClient('localhost', 27017)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [3]:
db = MongoClient().musicians.info

In [7]:
jackson = {'name': {'first': 'Michael',
                    'last': 'Jackson',
                    'middle': 'Joseph'},
           'born': datetime(1958, 8, 29),
           'died': datetime(2009, 6, 25),
           'albums': [{'name': 'Thriller',
                       'released': 1982},
                      {'name': 'Bad',
                       'released': 1987}]}

In [9]:
collection.drop()

In [10]:
result = db.insert_one(jackson)

In [11]:
result.acknowledged

True

In [12]:
result.inserted_id

ObjectId('588c576564ed26b558ff93f7')

In [13]:
db.find_one(result.inserted_id)

{u'_id': ObjectId('588c576564ed26b558ff93f7'),
 u'albums': [{u'name': u'Thriller', u'released': 1982},
  {u'name': u'Bad', u'released': 1987}],
 u'born': datetime.datetime(1958, 8, 29, 0, 0),
 u'died': datetime.datetime(2009, 6, 25, 0, 0),
 u'name': {u'first': u'Michael', u'last': u'Jackson', u'middle': u'Joseph'}}

In [14]:
db.find_one()

{u'_id': ObjectId('588c576564ed26b558ff93f7'),
 u'albums': [{u'name': u'Thriller', u'released': 1982},
  {u'name': u'Bad', u'released': 1987}],
 u'born': datetime.datetime(1958, 8, 29, 0, 0),
 u'died': datetime.datetime(2009, 6, 25, 0, 0),
 u'name': {u'first': u'Michael', u'last': u'Jackson', u'middle': u'Joseph'}}

In [16]:
db.find()

In [17]:
list(db.find().limit(10))

[{u'_id': ObjectId('588c576564ed26b558ff93f7'),
  u'albums': [{u'name': u'Thriller', u'released': 1982},
   {u'name': u'Bad', u'released': 1987}],
  u'born': datetime.datetime(1958, 8, 29, 0, 0),
  u'died': datetime.datetime(2009, 6, 25, 0, 0),
  u'name': {u'first': u'Michael', u'last': u'Jackson', u'middle': u'Joseph'}}]

In [21]:
db.find_one({'born': datetime(1958,8,29)})

{u'_id': ObjectId('588c576564ed26b558ff93f7'),
 u'albums': [{u'name': u'Thriller', u'released': 1982},
  {u'name': u'Bad', u'released': 1987}],
 u'born': datetime.datetime(1958, 8, 29, 0, 0),
 u'died': datetime.datetime(2009, 6, 25, 0, 0),
 u'name': {u'first': u'Michael', u'last': u'Jackson', u'middle': u'Joseph'}}

In [24]:
db.find_one({'name.first': 'Michael'})

{u'_id': ObjectId('588c576564ed26b558ff93f7'),
 u'albums': [{u'name': u'Thriller', u'released': 1982},
  {u'name': u'Bad', u'released': 1987}],
 u'born': datetime.datetime(1958, 8, 29, 0, 0),
 u'died': datetime.datetime(2009, 6, 25, 0, 0),
 u'name': {u'first': u'Michael', u'last': u'Jackson', u'middle': u'Joseph'}}

In [25]:
db.find_one({'albums.released': 1982})

{u'_id': ObjectId('588c576564ed26b558ff93f7'),
 u'albums': [{u'name': u'Thriller', u'released': 1982},
  {u'name': u'Bad', u'released': 1987}],
 u'born': datetime.datetime(1958, 8, 29, 0, 0),
 u'died': datetime.datetime(2009, 6, 25, 0, 0),
 u'name': {u'first': u'Michael', u'last': u'Jackson', u'middle': u'Joseph'}}

In [27]:
db.find_one({'albums.released': {'$gt': 2000}})

In [28]:
db.find_one({'albums.released': {'$gt': 1985}})

{u'_id': ObjectId('588c576564ed26b558ff93f7'),
 u'albums': [{u'name': u'Thriller', u'released': 1982},
  {u'name': u'Bad', u'released': 1987}],
 u'born': datetime.datetime(1958, 8, 29, 0, 0),
 u'died': datetime.datetime(2009, 6, 25, 0, 0),
 u'name': {u'first': u'Michael', u'last': u'Jackson', u'middle': u'Joseph'}}

In [30]:
# AND
db.find_one({'name.first': 'Michael', 'name.last': 'Bowie'})

In [32]:
db.find_one({'$or': [{'name.first': 'Michael'},
                             {'name.last': 'Bowie'}]})

{u'_id': ObjectId('588c576564ed26b558ff93f7'),
 u'albums': [{u'name': u'Thriller', u'released': 1982},
  {u'name': u'Bad', u'released': 1987}],
 u'born': datetime.datetime(1958, 8, 29, 0, 0),
 u'died': datetime.datetime(2009, 6, 25, 0, 0),
 u'name': {u'first': u'Michael', u'last': u'Jackson', u'middle': u'Joseph'}}

In [34]:
bowie = {'name': {'first': 'David',
                  'middle': 'Robert',
                  'last': 'Bowie'},
         'born': datetime(1947, 1, 8)}

In [35]:
db.insert_one(bowie)

In [36]:
list(db.find({}, {'name.last': 1, 'born': 1}))

[{u'_id': ObjectId('588c576564ed26b558ff93f7'),
  u'born': datetime.datetime(1958, 8, 29, 0, 0),
  u'name': {u'last': u'Jackson'}},
 {u'_id': ObjectId('588c5a3064ed26b558ff93f8'),
  u'born': datetime.datetime(1947, 1, 8, 0, 0),
  u'name': {u'last': u'Bowie'}}]

In [38]:
list(db.find({}, {'name.last': 1, 'born': 1}).sort('born'))

[{u'_id': ObjectId('588c5a3064ed26b558ff93f8'),
  u'born': datetime.datetime(1947, 1, 8, 0, 0),
  u'name': {u'last': u'Bowie'}},
 {u'_id': ObjectId('588c576564ed26b558ff93f7'),
  u'born': datetime.datetime(1958, 8, 29, 0, 0),
  u'name': {u'last': u'Jackson'}}]

In [45]:
from pymongo import DESCENDING

In [75]:
list(db.find({}, {'name.last': 1, 'born': 1})\
     .sort([('born', DESCENDING)]))

[{u'_id': ObjectId('588c576564ed26b558ff93f7'),
  u'born': datetime.datetime(1958, 8, 29, 0, 0),
  u'name': {u'last': u'Jackson'}},
 {u'_id': ObjectId('588c5a3064ed26b558ff93f8'),
  u'born': datetime.datetime(1947, 1, 8, 0, 0),
  u'name': {u'last': u'Bowie'}}]

## Modifying things

In [49]:
result = db.update_one({'name.last': 'Bowie'},
                       {'$set': {'albums': []}})

In [50]:
result.matched_count

1

In [51]:
result.modified_count

0

In [52]:
result = db.update_one({'name.last': 'Bowie'},
                       {'$push': {'albums': {'name': "Let's Dance",
                                             'released': 1983}}})

In [53]:
result.raw_result

{u'n': 1, u'nModified': 1, u'ok': 1.0, 'updatedExisting': True}

In [54]:
db.find_one({'name.last': 'Bowie'})

{u'_id': ObjectId('588c5a3064ed26b558ff93f8'),
 u'albums': [{u'name': u"Let's Dance", u'released': 1983}],
 u'born': datetime.datetime(1947, 1, 8, 0, 0),
 u'name': {u'first': u'David', u'last': u'Bowie', u'middle': u'Robert'}}

In [55]:
db.update_one({'name.last': 'Bowie'}, {'$inc': {'albums.0.released': 1}})

In [56]:
db.find_one({'name.last': 'Bowie'}, {'albums': 1})

{u'_id': ObjectId('588c5a3064ed26b558ff93f8'),
 u'albums': [{u'name': u"Let's Dance", u'released': 1984}]}

In [57]:
db.update_one({'name.last': 'Bowie','albums.name': "Let's Dance"},
              {'$inc': {'albums.$.released': -1}})

In [58]:
db.find_one({'name.last': 'Bowie'}, {'albums': 1})

{u'_id': ObjectId('588c5a3064ed26b558ff93f8'),
 u'albums': [{u'name': u"Let's Dance", u'released': 1983}]}

# Aggregations

In [62]:
agg = db.aggregate([{'$group': {'_id': '$born',
                                'count': {'$sum': 1}}}])

In [63]:
list(agg)

[{u'_id': datetime.datetime(1947, 1, 8, 0, 0), u'count': 1},
 {u'_id': datetime.datetime(1958, 8, 29, 0, 0), u'count': 1}]

In [68]:
list(db.aggregate([{'$group': {'_id': {'$year': '$born'},
                               'count': {'$sum': 1}}}]))

[{u'_id': 1947, u'count': 1}, {u'_id': 1958, u'count': 1}]

In [71]:
list(db.aggregate([{'$group': {'_id': {'$year': '$born'}, 
                               'max_records_released' :
                                 {'$max': {'$size': '$albums'}}}}]))

[{u'_id': 1947, u'max_records_released': 1},
 {u'_id': 1958, u'max_records_released': 2}]

In [74]:
list(db.aggregate([{'$match': {'name.first': 'David'}},
                   {'$group': {'_id': {'$year': '$born'}, 
                               'max_records_released' : 
                                 {'$max': {'$size': '$albums'}}}}]))

[{u'_id': 1947, u'max_records_released': 1}]

In [5]:
list(db.aggregate([{'$project': {'last_record': {'$max': '$albums.released'}}}]))

[{u'_id': ObjectId('588c576564ed26b558ff93f7'), u'last_record': 1987},
 {u'_id': ObjectId('588c5a3064ed26b558ff93f8'), u'last_record': 1983}]

In [6]:
from bson import Code

In [48]:
map_fn = Code("""function() {emit(this.born.getFullYear(), this.albums.length)}""")
reduce_fn = Code('function(key, values) {return Array.sum(values)} ')

In [49]:
result = db.map_reduce(map_fn, reduce_fn, 'result')

In [50]:
list(result.find())

[{u'_id': 1947.0, u'value': 1.0}, {u'_id': 1958.0, u'value': 2.0}]